In [1]:
%load_ext cython

In [2]:
import random
import collections
import itertools
import numpy as np

In [6]:
x = np.array(list(itertools.product(range(31),range(31),range(31))))
x

array([[ 0,  0,  0],
       [ 0,  0,  1],
       [ 0,  0,  2],
       ...,
       [30, 30, 28],
       [30, 30, 29],
       [30, 30, 30]])

In [7]:
x2 = x[np.sum(x, 1 ) == 31]

In [8]:
x2.shape

(525, 3)

In [15]:
1*1/2

0.5

In [16]:
(1/4,4/16*3/4)*2

0.875

In [17]:
(4/16*3/4,8/64*3/4*3/4)*3

0.7734375

In [14]:
1/4,4/16*3/4, 2

0.4375

In [12]:
n,m = 20000000, 3*2
x = np.random.randint(0,2,[n, m])
print(x,"\n")
for i in range(0,m-1, 2):
    # print(i,i,2)
    x[np.sum(x[:,i:i,2], 1) == 0] = np.zeros(m)
# print(x)
np.mean(np.sum(x,1))

[[0 0 1 1 1 0]
 [1 1 1 0 1 0]
 [0 0 0 1 0 1]
 ...
 [1 0 0 1 0 1]
 [0 1 1 0 0 1]
 [0 0 0 1 0 0]] 



1.6872415

# old

In [10]:
import random
import numpy as np
import collections
from mpire import WorkerPool
max_points = 31

class Player():
    def __init__(self, parameters) -> None:
        self.hand = 0
        self.points = 0
        self.strategy = parameters

    def set_hand(self, n):
        self.hand = n
    
    def add_to_hand(self, n):
        self.hand += n
        
    def get_points(self):
        return self.points
    
    def get_hand(self):
        return self.hand
    
    def add_points(self, n):
        self.points += n
        
    def sub_points(self, n):
        self.points -=n
    
    def calc_weights(self, x, n, k, gamma, delta):
        return np.exp(x[0] + x[1]*delta + x[2]*delta**2/n + x[3]*delta**3/n**2 + x[4]*gamma + x[5]*gamma**2/n + x[6]*gamma**3/n**2 , dtype = np.float128)
    
    def get_p(self, strategy, oponent_points, pile):
        n =  max_points - self.points
        gamma = min(n-oponent_points, 0)
        delta = max(n-oponent_points, 0)
        weights = np.apply_along_axis(self.calc_weights, 1, strategy, n, pile, gamma, delta)
        return weights/np.sum(weights)

class rrrats4_player(Player):
    def make_decision_about_throw(self, oponent_points, pile):
        weights = self.get_p(self.strategy[:4,:], oponent_points, pile)
        choice = random.choices([1, 2, 3, 4], weights = weights)[0]
        if self.hand < choice:
            return 1 #play next move
        else:
            return 0
        
def tour(player_1, player_2, pile):
    for i in range(4):
        tour_results = player_1.make_decision_about_throw(player_2.get_points(), pile)
        print("tour_results: ",tour_results)
        if tour_results == 1:
            draw = [random.randint(0,1), random.randint(0,1)]
            if i == 0:
                draw = [1,0]
            if i == 1:
                draw = [1,0]
            if i == 2:
                draw = [1,0]
            if i == 3:
                draw = [1,1]
            print("draw: ",draw)
            if draw == [0,0]:
                player_1.set_hand(0)
                break
            else:
                player_1.add_to_hand(sum(draw))
        else:
            break

def symulation(parameters1, parameters2):
    np.random.seed()
    pile = max_points
    player_1 = rrrats4_player(parameters1)
    player_2 = rrrats4_player(parameters2)
    print("pile: ", pile)

    while True:
        tour_results = tour(player_1, player_2, pile)
        player_1_hand = player_1.get_hand()
        if player_1.get_hand() > 4:
            pile += player_1.get_hand() - 4
            player_1.set_hand(4)
        print("pile: ", pile)

        pile -= player_1.get_hand()
        print("player_1.get_hand(): ",player_1.get_hand())
        print("pile: ", pile)

        break
    
par = np.array([[0,1,2,3,4,5,6],
                [1,0,0,0,0,0,0], 
                [1,0,0,0,0,0,0],
                [10,0,0,0,0,0,]])

symulation(par, par)
pass

/tmp/ipykernel_87605/2427049164.py:94: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  par = np.array([[0,1,2,3,4,5,6],


pile:  31


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

# new

In [137]:
import random
import numpy as np
import collections
from mpire import WorkerPool
max_points = 31

class Player():
    def __init__(self, strategy) -> None:
        self.hand = 0
        self.points = 0
        self.strategy = strategy

    def set_hand(self, n):
        self.hand = n
    
    def add_to_hand(self, n):
        self.hand += n
        
    def get_points(self):
        return self.points
    
    def get_hand(self):
        return self.hand
    
    def add_points(self, n):
        self.points += n
        
    def sub_points(self, n):
        self.points -=n
    
    def calc_weights(self, x, n, pile, gamma, delta):
        # return np.exp(x[0] + x[1]*delta + x[2]*delta**2/n + x[3]*delta**3/n**2 + x[4]*gamma + x[5]*gamma**2/n + x[6]*gamma**3/n**2 , dtype = np.float128)
        return np.exp(x[0] + x[1]*delta/pile + x[2]*delta**2/pile**2 + x[3]*gamma/pile + x[4]*gamma**2/pile**2  , dtype = np.float128)

        
    def get_p(self, strategy, oponent_points, pile):
        n =  max_points - self.points
        gamma = min(n-oponent_points, 0)
        delta = max(n-oponent_points, 0)
        weights = np.apply_along_axis(self.calc_weights, 1, strategy, n, pile, gamma, delta)
        print("pile: ", pile, "weights: ", weights)
        # return weights/np.sum(weights)
        return weights
    
class rrrats4_player(Player):
    # def make_decision_about_throw(self, oponent_points, pile):
    #     weights = self.get_p(self.strategy, oponent_points, pile)
    #     choice = random.choices([1, 2, 3, 4], weights = weights)[0]
    #     print("choice :", choice)
    #     if self.hand < choice:
    #         return 1 #play next move
    #     else:
    #         return 0
    def make_decision_about_throw(self, oponent_points, pile):
        hand = self.hand
        if hand == 0:
            return 1
        elif hand == 1:
            weights = self.get_p(self.strategy[:2, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]
        elif hand == 2:
            weights = self.get_p(self.strategy[2:4, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]
        else:
            weights = self.get_p(self.strategy[4:, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]    
        print("choice :", choice, "weights: ", weights, "hand: ", hand)
        return choice

def tour(player_1, player_2, pile):
    for i in range(4):
        tour_results = player_1.make_decision_about_throw(player_2.get_points(), pile)
        print("Play next move: ", tour_results)
        if tour_results == 1:
            draw = [random.choices([0, 1, 2], weights = [3, 2, 1])[0], random.choices([0, 1, 2], weights = [3, 2, 1])[0]]
            print("Drow: ", draw)
            
            if draw == [0,0]:
                pile += player_1.get_hand()
                player_1.set_hand(0)
                break
            
            if draw[0] == 1:
                player_1.add_to_hand(1)
                pile -= 1
            elif draw[0] == 2:
                if player_2.get_points() > 0: #still from oponent
                    player_2.sub_points(1)
                    player_1.add_to_hand(1)
                else:
                    player_1.add_to_hand(1)
                    pile -= 1
            else:
                pass
            
            if draw[1] == 1:
                player_1.add_to_hand(1)
                pile -= 1
            elif draw[1] == 2:
                if player_2.get_points() > 0: #still from oponent
                    player_2.sub_points(1)
                    player_1.add_to_hand(1)
                else:
                    player_1.add_to_hand(1)
                    pile -= 1
            else:
                pass
            
            print("pile:", pile, "after: ", i)
            print("player_1 hand: ", player_1.get_hand(), "player_2 hand: ", player_2.get_hand())
            if pile < 1:
                break
        else:
            break
    print("player_1 hand end: ", player_1.get_hand())
    if player_1.get_hand() > 4:
        pile +=  player_1.get_hand() - 4
        player_1.set_hand(4)
    player_1.add_points(player_1.get_hand())
    player_1.set_hand(0)
    return pile

def symulation(parameters1, parameters2):
    parameters1 = np.reshape(parameters1, (6,5))
    parameters2 = np.reshape(parameters2, (6,5))
    np.random.seed()
    pile = max_points
    player_1 = rrrats4_player(parameters1)
    player_2 = rrrats4_player(parameters2)

    while True:
        # try:
            print("Player 1 tour")
            pile = tour(player_1, player_2, pile)
            print("piel: ", pile)
            print("new_pile after 1: ", pile, "player_1 points: ", player_1.get_points(), "player_2 points: ", player_2.get_points())
            
            print("Player 2 tour")
            pile = tour(player_2, player_1, pile)
            print("new_pile after 1: ", pile, "player_1 points: ", player_1.get_points(), "player_2 points: ", player_2.get_points())
            if pile < 1:
                break
        # except ValueError: 
            # print("hahahahah")
            # break
    if player_1.get_points() > player_2.get_points():
        return 1
    else:
        return 0
    
par = np.array([[100,100,100,100,100,
                1,0,0,0,0, 
                1,0,0,0,0,
                1,0,0,0,0,
                1,0,0,0,0,
                1,0,0,0,0]])

print("winner: ", symulation(par, par))
pass

TypeError: _reshape_dispatcher() got an unexpected keyword argument 'dtype'

# end preper

In [142]:
import random
import numpy as np
import collections
from mpire import WorkerPool
max_points = 31

class Player():
    def __init__(self, strategy) -> None:
        self.hand = 0
        self.points = 0
        self.strategy = strategy

    def set_hand(self, n):
        self.hand = n
    
    def add_to_hand(self, n):
        self.hand += n
        
    def get_points(self):
        return self.points
    
    def get_hand(self):
        return self.hand
    
    def add_points(self, n):
        self.points += n
        
    def sub_points(self, n):
        self.points -=n
    
    def calc_weights(self, x, n, pile, gamma, delta):
        # return np.exp(x[0] + x[1]*delta + x[2]*delta**2/n + x[3]*delta**3/n**2 + x[4]*gamma + x[5]*gamma**2/n + x[6]*gamma**3/n**2 , dtype = np.float128)
        return np.exp(x[0] + x[1]*delta/pile + x[2]*delta**2/pile**2 + x[3]*gamma/pile + x[4]*gamma**2/pile**2  , dtype = np.float128)

        
    def get_p(self, strategy, oponent_points, pile):
        n =  max_points - self.points
        gamma = min(n-oponent_points, 0)
        delta = max(n-oponent_points, 0)
        weights = np.apply_along_axis(self.calc_weights, 1, strategy, n, pile, gamma, delta)
        # return weights/np.sum(weights)
        return weights
    
class rrrats4_player(Player):
    def make_decision_about_throw(self, oponent_points, pile):
        hand = self.hand
        if hand == 0:
            return 1
        elif hand == 1:
            weights = self.get_p(self.strategy[:2, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]
        elif hand == 2:
            weights = self.get_p(self.strategy[2:4, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]
        else:
            weights = self.get_p(self.strategy[4:, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]    
        return choice

def tour(player_1, player_2, pile):
    for i in range(4):
        tour_results = player_1.make_decision_about_throw(player_2.get_points(), pile)
        if tour_results == 1:
            draw = [random.choices([0, 1, 2], weights = [3, 2, 1])[0], random.choices([0, 1, 2], weights = [3, 2, 1])[0]]
            
            if draw == [0,0]:
                pile += player_1.get_hand()
                player_1.set_hand(0)
                break
            
            if draw[0] == 1:
                player_1.add_to_hand(1)
                pile -= 1
            elif draw[0] == 2:
                if player_2.get_points() > 0: #still from oponent
                    player_2.sub_points(1)
                    player_1.add_to_hand(1)
                else:
                    player_1.add_to_hand(1)
                    pile -= 1
            else:
                pass
            
            if draw[1] == 1:
                player_1.add_to_hand(1)
                pile -= 1
            elif draw[1] == 2:
                if player_2.get_points() > 0: #still from oponent
                    player_2.sub_points(1)
                    player_1.add_to_hand(1)
                else:
                    player_1.add_to_hand(1)
                    pile -= 1
            else:
                pass
            if pile < 1:
                break
        else:
            break
    if player_1.get_hand() > 4:
        pile +=  player_1.get_hand() - 4
        player_1.set_hand(4)
    player_1.add_points(player_1.get_hand())
    player_1.set_hand(0)
    return pile

def symulation(parameters1, parameters2):
    parameters1 = np.reshape(parameters1, (6,5))
    parameters2 = np.reshape(parameters2, (6,5))
    np.random.seed()
    pile = max_points
    player_1 = rrrats4_player(parameters1)
    player_2 = rrrats4_player(parameters2)

    while True:
        # try:

            pile = tour(player_1, player_2, pile)
            pile = tour(player_2, player_1, pile)
            if pile < 1:
                break
        # except ValueError: 
            # print("hahahahah")
            # break
    if player_1.get_points() > player_2.get_points():
        return 1
    else:
        return 0
    
par = np.array([[100,100,100,100,100,
                1,0,0,0,0, 
                1,0,0,0,0,
                1,0,0,0,0,
                1,0,0,0,0,
                1,0,0,0,0]])

print("winner: ", symulation(par, par))
pass

winner:  0


In [143]:
%%timeit
symulation(par, par)

3.04 ms ± 288 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# cython


In [6]:
%%cython
import random
import numpy as np
cimport numpy as np
np.import_array()

import collections
from mpire import WorkerPool
cdef short max_points
max_points = 31

# cdef double parameters_0[3]
# cdef int games_to_play
cdef class Player():
    cdef short hand, points
    cdef np.ndarray strategy
    def __init__(self, strategy) -> None:
        self.hand = 0
        self.points = 0
        self.strategy = strategy

    def set_hand(self, n):
        self.hand = n
    
    def add_to_hand(self, n):
        self.hand += n
        
    def get_points(self):
        return self.points
    
    def get_hand(self):
        return self.hand
    
    def add_points(self, n):
        self.points += n
        
    def sub_points(self, n):
        self.points -=n
    
    def calc_weights(self, x, n, pile, gamma, delta):
        # return np.exp(x[0] + x[1]*delta + x[2]*delta**2/n + x[3]*delta**3/n**2 + x[4]*gamma + x[5]*gamma**2/n + x[6]*gamma**3/n**2 , dtype = np.float128)
        return np.exp(x[0] + x[1]*delta/pile + x[2]*delta**2/pile**2 + x[3]*gamma/pile + x[4]*gamma**2/pile**2  , dtype = np.float128)

        
    def get_p(self, strategy, oponent_points, pile):
        n =  max_points - self.points
        gamma = min(n-oponent_points, 0)
        delta = max(n-oponent_points, 0)
        weights = np.apply_along_axis(self.calc_weights, 1, strategy, n, pile, gamma, delta)
        return weights
    
cdef class rrrats4_player(Player):
    def make_decision_about_throw(self, oponent_points, pile):
        if self.hand == 0:
            return 1
        elif self.hand == 1:
            weights = self.get_p(self.strategy[:2, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]
        elif self.hand == 2:
            weights = self.get_p(self.strategy[2:4, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]
        else:
            weights = self.get_p(self.strategy[4:, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]    
        return choice

def tour(player_1, player_2, pile):
    for i in range(4):
        tour_results = player_1.make_decision_about_throw(player_2.get_points(), pile)
        if tour_results == 1:
            draw = [random.choices([0, 1, 2], weights = [3, 2, 1])[0], random.choices([0, 1, 2], weights = [3, 2, 1])[0]]            
            if draw == [0,0]:
                pile += player_1.get_hand()
                player_1.set_hand(0)
                break
            
            if draw[0] == 1:
                player_1.add_to_hand(1)
                pile -= 1
            elif draw[0] == 2:
                if player_2.get_points() > 0: #still from oponent
                    player_2.sub_points(1)
                    player_1.add_to_hand(1)
                else:
                    player_1.add_to_hand(1)
                    pile -= 1
            else:
                pass
            
            if draw[1] == 1:
                player_1.add_to_hand(1)
                pile -= 1
            elif draw[1] == 2:
                if player_2.get_points() > 0: #still from oponent
                    player_2.sub_points(1)
                    player_1.add_to_hand(1)
                else:
                    player_1.add_to_hand(1)
                    pile -= 1
            else:
                pass

            if pile < 1:
                break
        else:
            break
        
    if player_1.get_hand() > 4:
        pile +=  player_1.get_hand() - 4
        player_1.set_hand(4)
    player_1.add_points(player_1.get_hand())
    player_1.set_hand(0)
    return pile

def symulation(parameters1, parameters2):
    cdef short pile
    np.random.seed()
    
    parameters1 = np.reshape(parameters1, (6,5))
    parameters2 = np.reshape(parameters2, (6,5))
    
    pile = max_points
    player_1 = rrrats4_player(parameters1)
    player_2 = rrrats4_player(parameters2)

    while True:
        # try:
            pile = tour(player_1, player_2, pile)
            pile = tour(player_2, player_1, pile)
            if pile < 1:
                break
        # except ValueError: 
            # print("hahahahah")
            # break
    if player_1.get_points() > player_2.get_points():
        return 1
    else:
        return 0
    


0

In [247]:
%%cython
import random
import numpy as np
cimport numpy as np
np.import_array()

import collections
from mpire import WorkerPool
cdef short max_points
max_points = 31


cdef class Player():
    cdef short hand, points
    cdef np.ndarray strategy
    def __init__(self, strategy) -> None:
        self.hand = 0
        self.points = 0
        self.strategy = strategy

    def set_hand(self, n):
        self.hand = n
    
    def add_to_hand(self, n):
        self.hand += n
        
    def get_points(self):
        return self.points
    
    def get_hand(self):
        return self.hand
    
    def add_points(self, n):
        self.points += n
        
    def sub_points(self, n):
        self.points -=n
    
    def calc_weights(self, x, n, pile, gamma, delta):
        # return np.exp(x[0] + x[1]*delta + x[2]*delta**2/n + x[3]*delta**3/n**2 + x[4]*gamma + x[5]*gamma**2/n + x[6]*gamma**3/n**2 , dtype = np.float128)
        return np.exp(x[0] + x[1]*delta/pile + x[2]*delta**2/pile**2 + x[3]*gamma/pile + x[4]*gamma**2/pile**2  , dtype = np.float128)

        
    def get_p(self, strategy, oponent_points, pile):
        n =  max_points - self.points
        gamma = min(n-oponent_points, 0)
        delta = max(n-oponent_points, 0)
        weights = np.apply_along_axis(self.calc_weights, 1, strategy, n, pile, gamma, delta)
        return weights
    
cdef class rrrats4_player(Player):
    def make_decision_about_throw(self, oponent_points, pile):
        if self.hand == 0:
            return 1
        elif self.hand == 1:
            weights = self.get_p(self.strategy[:2, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]
        elif self.hand == 2:
            weights = self.get_p(self.strategy[2:4, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]
        else:
            weights = self.get_p(self.strategy[4:, :], oponent_points, pile)
            choice = random.choices([1, 0], weights = weights)[0]    
        return choice

def tour(player_1, player_2, pile):
    for i in range(4):
        tour_results = player_1.make_decision_about_throw(player_2.get_points(), pile)
        if tour_results == 1:
            draw = [random.choices([0, 1, 2], weights = [3, 2, 1])[0], random.choices([0, 1, 2], weights = [3, 2, 1])[0]]            
            if draw == [0,0]:
                pile += player_1.get_hand()
                player_1.set_hand(0)
                break
            
            if draw[0] == 1:
                player_1.add_to_hand(1)
                pile -= 1
            elif draw[0] == 2:
                if player_2.get_points() > 0: #still from oponent
                    player_2.sub_points(1)
                    player_1.add_to_hand(1)
                else:
                    player_1.add_to_hand(1)
                    pile -= 1
            else:
                pass
            if pile < 1:
                break
            if draw[1] == 1:
                player_1.add_to_hand(1)
                pile -= 1
            elif draw[1] == 2:
                if player_2.get_points() > 0: #still from oponent
                    player_2.sub_points(1)
                    player_1.add_to_hand(1)
                else:
                    player_1.add_to_hand(1)
                    pile -= 1
            else:
                pass

            if pile < 1:
                break
        else:
            break
        
    if player_1.get_hand() > 4:
        pile +=  player_1.get_hand() - 4
        player_1.set_hand(4)
    player_1.add_points(player_1.get_hand())
    player_1.set_hand(0)
    return pile

def symulation(parameters1, parameters2):
    cdef short pile
    np.random.seed()
    parameters1 = np.reshape(parameters1, (6,5))
    parameters2 = np.reshape(parameters2, (6,5))
    
    pile = max_points
    player_1 = rrrats4_player(parameters1)
    player_2 = rrrats4_player(parameters2)
    
    while True:
        # try:
            pile = tour(player_1, player_2, pile)
            if pile < 1:
                break
            pile = tour(player_2, player_1, pile)
            if pile < 1:
                break

        # except ValueError: 
            # print("hahahahah")
            # break
    if player_1.get_points() > player_2.get_points():
        return 1
    else:
        return 0
    


In [248]:
def calc_weights(x, n, pile, gamma, delta):
        # return np.exp(x[0] + x[1]*delta + x[2]*delta**2/n + x[3]*delta**3/n**2 + x[4]*gamma + x[5]*gamma**2/n + x[6]*gamma**3/n**2 , dtype = np.float128)
        return np.exp(x[0] + x[1]*delta/pile + x[2]*delta**2/pile**2 + x[3]*gamma/pile + x[4]*gamma**2/pile**2  , dtype = np.float128)

def get_p(strategy, points, oponent_points, pile):
        n =  31 - points
        gamma = min(n-oponent_points, 0)
        delta = max(n-oponent_points, 0)
        weights = np.apply_along_axis(calc_weights, 1, strategy, n, pile, gamma, delta)
        return weights

In [256]:
n,m = 0, 0
k = 31 - n-m
par1 = np.array([[10,0,0,0,0,
                0,0,0,0,0, 
                10,0,0,0,0,
                0,0,0,0,0,
                0,0,0,0,0,
                1,0,0,0,0]], dtype=np.float128)
par1 = np.reshape(par1,((6,5)))
ps = get_p(par1, n, m, k)
print(ps[0]/(ps[0]+ps[1]), ps[2]/(ps[2]+ps[3]), ps[4]/(ps[4]+ps[5]))

par2 = np.array([[10,0,0,0,0,
                0,0,0,0,0, 
                10,0,0,0,0,
                0,0,0,0,0,
                0,0,0,0,0,
                10,0,0,0,0]], dtype=np.float128)
par2 = np.reshape(par2,((6,5)))
ps = get_p(par2, n, m, k)
print(ps[0]/(ps[0]+ps[1]), ps[2]/(ps[2]+ps[3]), ps[4]/(ps[4]+ps[5]))
np.mean([symulation(par1, par2) for i in range(2000)])


0.9999546021312975656 0.9999546021312975656 0.26894142136999512076
0.9999546021312975656 0.9999546021312975656 4.5397868702434394504e-05


0.516

0.75

In [153]:
symulation(par1, par2)

(0, 13, 18)

In [145]:
symulation(par1, par1)


(1, 18, 13)

In [84]:
%%timeit
symulation(par,par)

2.54 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# actual 

In [227]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


UsageError: %%cython is a cell magic, but the cell body is empty.


In [242]:
%%cython
import random
import numpy as np
cimport numpy as np
np.import_array()

import collections
from mpire import WorkerPool
cdef short max_points
max_points = 31


cdef class Player():
    cdef short hand, points
    cdef np.ndarray strategy
    def __init__(self, strategy) -> None:
        self.hand = 0
        self.points = 0
        self.strategy = np.exp(strategy, dtype = np.float128)

    def set_hand(self, n):
        self.hand = n
    
    def add_to_hand(self, n):
        self.hand += n
        
    def get_points(self):
        return self.points
    
    def get_hand(self):
        return self.hand
    
    def add_points(self, n):
        self.points += n
        
    def sub_points(self, n):
        self.points -=n
    
    def calc_weights(self, x, n, pile, gamma, delta):
        # return np.exp(x[0] + x[1]*delta + x[2]*delta**2/n + x[3]*delta**3/n**2 + x[4]*gamma + x[5]*gamma**2/n + x[6]*gamma**3/n**2 , dtype = np.float128)
        # return np.exp(x[0] + x[1]*delta/pile + x[2]*delta**2/pile**2 + x[3]*gamma/pile + x[4]*gamma**2/pile**2  , dtype = np.float128)
        return np.exp(x[0], dtype = np.float128)


    def get_p(self, strategy, oponent_points, pile):
        n =  max_points - self.points
        gamma = min(n-oponent_points, 0)
        delta = max(n-oponent_points, 0)
        weights = np.apply_along_axis(self.calc_weights, 1, strategy, n, pile, gamma, delta)
        return weights
    
cdef class rrrats4_player(Player):
    def make_decision_about_throw(self, oponent_points, pile):
        if self.hand == 0:
            return 1
        elif self.hand == 1:
            choice = random.choices([1, 0], weights = self.strategy[:2])[0]
        elif self.hand == 2:
            choice = random.choices([1, 0], weights = self.strategy[2:4])[0]
        else:
            choice = random.choices([1, 0], weights = self.strategy[4:])[0]
        return choice

def tour(player_1, player_2, pile):
    for i in range(4):
        tour_results = player_1.make_decision_about_throw(player_2.get_points(), pile)
        if tour_results == 1:
            draw = [random.choices([0, 1, 2], weights = [3, 2, 1])[0], random.choices([0, 1, 2], weights = [3, 2, 1])[0]]            
            if draw == [0,0]:
                pile += player_1.get_hand()
                player_1.set_hand(0)
                break
            
            if draw[0] == 1:
                player_1.add_to_hand(1)
                pile -= 1
            elif draw[0] == 2:
                if player_2.get_points() > 0: #still from oponent
                    player_2.sub_points(1)
                    player_1.add_to_hand(1)
                else:
                    player_1.add_to_hand(1)
                    pile -= 1
            else:
                pass
            if pile < 1:
                break
            if draw[1] == 1:
                player_1.add_to_hand(1)
                pile -= 1
            elif draw[1] == 2:
                if player_2.get_points() > 0: #still from oponent
                    player_2.sub_points(1)
                    player_1.add_to_hand(1)
                else:
                    player_1.add_to_hand(1)
                    pile -= 1
            else:
                pass

            if pile < 1:
                break
        else:
            break
        
    if player_1.get_hand() > 4:
        pile +=  player_1.get_hand() - 4
        player_1.set_hand(4)
    player_1.add_points(player_1.get_hand())
    player_1.set_hand(0)
    return pile

def symulation(parameters1, parameters2):
    cdef short pile
    np.random.seed()
    
    pile = max_points
    player_1 = rrrats4_player(parameters1)
    player_2 = rrrats4_player(parameters2)
    
    while True:
        # try:
            pile = tour(player_1, player_2, pile)
            if pile < 1:
                break
            pile = tour(player_2, player_1, pile)
            if pile < 1:
                break

        # except ValueError: 
            # print("hahahahah")
            # break
    if player_1.get_points() > player_2.get_points():
        return 1
    else:
        return 0

In file included from /home/olek/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1948,
                 from /home/olek/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/olek/.local/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/olek/.cache/ipython/cython/_cython_magic_feaead3dd09770b58bfc6f3a099dad14.c:794:
/home/olek/.local/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


In [244]:
par1 = np.array([10,0,0,0,0,0])
par2 = np.array([0,0,0,0,0,0])
np.mean([symulation(par1, par2) for i in range(100)])

0.72

In [246]:
%%timeit 
symulation(par1, par2)

493 µs ± 6.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
